In [1]:
import os
import sys

from datasets import load_dataset

from tqdm import tqdm
from matplotlib import pyplot as plt
import pretty_midi
import pandas as pd
import glob
import numpy as np
import muspy
import pypianoroll
import torch
from util.play_midi import play_midi

import miditok
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import DataLoader

import transformers
import huggingface_hub
import tokenizers

In [6]:
# os.chdir("..")
print(os.getcwd())
print(os.curdir)

midi_files = list(Path("nottingham-dataset-master/nottingham-dataset-master/MIDI").glob("*.mid"))

c:\Users\joeac\Documents\Uni\Y3\Project
.


In [9]:
len(midi_files)
track_length = []
for i in tqdm(midi_files):
    track_length.append(len(Score(i).tracks))


100%|██████████| 1034/1034 [00:01<00:00, 926.13it/s]


In [15]:
for i,j in enumerate(track_length):
    if j != 2:
        print("noo")
        print(j, i)
        

noo
1 76
noo
1 374
noo
1 407
noo
1 471
noo
1 473
noo
1 476
noo
1 477
noo
1 478
noo
1 626
noo
1 672
noo
1 742
noo
1 879
noo
1 1006


In [18]:
play_midi(midi_files[1007])
# Score(midi_files[1006])

c:\Users\joeac\AppData\Local\Programs\Python\Python311\Lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [2]:
### how to load tokenizer 
midi_files = list(Path("data/").glob("**/*.mid"))

# os.chdir("..")
print(os.environ)


# tokeniizer_config = miditok.TokenizerConfig(use_chords=True,)
# tokenizer_remi = miditok.REMI(tokenizer_config=tokeniizer_config)

# tokenizer_download = huggingface_hub.hf_hub_download("theglassofwater/tokenizer_trained", filename="tokenizer.json") # puts file in .cache file
# tokenizer = tokenizers.Tokenizer.from_file(tokenizer_download) # from local file.
# fast_tokenizer = transformers.PreTrainedTokenizerFast(tokenizer_file=tokenizer_download) # from local

tokenizer = miditok.MIDITokenizer.from_pretrained("theglassofwater/tokenizer_trained") # from hub


environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\joeac\\AppData\\Roaming', 'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': '1', 'CABAL_DIR': 'C:\\cabal', 'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey', 'CHOCOLATEYLASTPATHUPDATE': '133474593169954457', 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_17768_TEVDHIOFLXYLTRDY', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'Y-EL-AASAR-LAPT', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'EFC_4540': '1', 'ELECTRON_RUN_AS_NODE': '1', 'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer', 'FPS_BROWSER_USER_PROFILE_STRING': 'Default', 'GHCUP_INSTALL_BASE_PREFIX': 'C:\\', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\joeac', 'JPY_INTERRUPT_EVENT': '1796', 'LOCALAPPDATA': 'C:\\Users\\joeac\\AppData\\Local', 'LOGONSERV

In [45]:
tokenizer.config

In [14]:
tokenizer.save_pretrained(save_directory="model/tokenizer_trained", push_to_hub=True, )

CommitInfo(commit_url='https://huggingface.co/theglassofwater/tokenizer_trained/commit/7773845a2b74b58827863ad208e55c8e136f5ffb', commit_message='Push model using huggingface_hub.', commit_description='', oid='7773845a2b74b58827863ad208e55c8e136f5ffb', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
tokenizer._create_token_types_graph()

{'Pitch': ['Velocity'],
 'Velocity': ['Duration'],
 'Duration': ['Pitch', 'Position', 'Bar', 'PitchDrum'],
 'Bar': ['Position', 'Bar'],
 'Position': ['Pitch', 'Chord', 'PitchDrum'],
 'Chord': ['Pitch', 'PitchDrum'],
 'PitchDrum': ['Velocity']}

In [9]:
tokenizer_config = miditok.TokenizerConfig(use_programs=True)
tokenizer_test = miditok.REMI(tokenizer_config)
tokenizer_test.tokens_types_graph

{'Program': ['Pitch', 'PitchDrum', 'PAD', 'EOS', 'MASK'],
 'Pitch': ['Velocity', 'PAD', 'PAD', 'EOS', 'EOS', 'MASK', 'MASK'],
 'Velocity': ['Duration', 'PAD', 'EOS', 'MASK'],
 'Duration': ['Program', 'Position', 'Bar', 'PAD', 'EOS', 'MASK'],
 'Bar': ['Position', 'Bar', 'PAD', 'EOS', 'MASK'],
 'Position': ['Program', 'PAD', 'EOS', 'MASK'],
 'PitchDrum': ['Velocity', 'PAD', 'PAD', 'EOS', 'EOS', 'MASK', 'MASK'],
 'PAD': ['Program',
  'Pitch',
  'Velocity',
  'Duration',
  'Bar',
  'Position',
  'PitchDrum',
  'PAD_None',
  'BOS_None',
  'EOS_None',
  'MASK_None'],
 'BOS': ['Program',
  'Pitch',
  'Velocity',
  'Duration',
  'Bar',
  'Position',
  'PitchDrum',
  'PAD_None',
  'BOS_None',
  'EOS_None',
  'MASK_None'],
 'EOS': [],
 'MASK': ['Program',
  'Pitch',
  'Velocity',
  'Duration',
  'Bar',
  'Position',
  'PitchDrum',
  'PAD_None',
  'BOS_None',
  'EOS_None',
  'MASK_None']}

In [4]:
x = tokenizer_test(midi_files[-2])

In [6]:
midi_files[-2]
play_midi(midi_files[-2])

In [43]:
x

TokSequence(tokens=['Bar_None', 'Position_0', 'Program_0', 'Pitch_72', 'Velocity_91', 'Duration_1.4.8', 'Program_0', 'Pitch_41', 'Velocity_91', 'Duration_4.0.4', 'Program_0', 'Pitch_45', 'Velocity_91', 'Duration_4.0.4', 'Program_0', 'Pitch_48', 'Velocity_91', 'Duration_4.0.4', 'Position_12', 'Program_0', 'Pitch_70', 'Velocity_91', 'Duration_0.4.8', 'Position_16', 'Program_0', 'Pitch_69', 'Velocity_91', 'Duration_1.0.8', 'Position_24', 'Program_0', 'Pitch_67', 'Velocity_91', 'Duration_1.0.8', 'Bar_None', 'Position_0', 'Program_0', 'Pitch_65', 'Velocity_91', 'Duration_1.0.8', 'Program_0', 'Pitch_41', 'Velocity_91', 'Duration_4.0.4', 'Program_0', 'Pitch_45', 'Velocity_91', 'Duration_4.0.4', 'Program_0', 'Pitch_48', 'Velocity_91', 'Duration_4.0.4', 'Position_8', 'Program_0', 'Pitch_67', 'Velocity_91', 'Duration_1.0.8', 'Position_16', 'Program_0', 'Pitch_69', 'Velocity_91', 'Duration_1.0.8', 'Position_24', 'Program_0', 'Pitch_65', 'Velocity_91', 'Duration_1.0.8', 'Bar_None', 'Position_0', '

https://github.com/briansemrau/MIDI-LLM-tokenizer   

In [13]:
import mido
import json
import random
from dataclasses import dataclass
from functools import lru_cache
from typing import Dict, Iterator, List, Optional, Tuple
from math import ceil, floor, log

class VocabConfig:
    # Number of note events. Should be 128.
    note_events: int
    # Number of wait events. Configurable, must evenly divide max_wait_time.
    wait_events: int
    # Max wait time in milliseconds to be represented by a single token.
    max_wait_time: int
    # Number of velocity events. Should be 128 (or 100? need to check midi standard)
    velocity_events: int
    # Number of bins to quantize velocity into. Should evenly divide velocity_events.
    velocity_bins: int
    # Exponential scaling factor for velocity bin sizes. 1.0 = linear scaling.
    velocity_exp: float
    # Whether to sort tokens by instrument, note. This should improve data reducibility.
    do_token_sorting: bool
    # Whether tokens should be represented as combined instrument/note/velocity tokens, or separate tokens for each.
    unrolled_tokens: bool
    # If non-zero, notes held for this many seconds will be automatically released during str->midi decoding.
    decode_end_held_note_delay: float
    # If true, repeated notes will be automatically released before playing again during str->midi decoding.
    decode_fix_repeated_notes: bool
    # List of instrument names to use for binning. Must have at most 16 values.
    bin_instrument_names: List[str]
    # Indicates which bin name represents percussion instruments on MIDI channel 10.
    ch10_instrument_bin_name: str
    # Mapping from instrument name to bin name.
    program_name_to_bin_name: Dict[str, str]
    # Mapping from bin name to program name.
    bin_name_to_program_name: Dict[str, str]
    # Mapping from program number to instrument name.
    instrument_names: Dict[str, str]
    # Manual override for velocity bins. Each element is the max velocity value for that bin by index.
    velocity_bins_override: Optional[List[int]] = None

    def __post_init__(self):
        self.validate()
        
        self._instrument_names_str_to_int = {name: int(i) for i, name in self.instrument_names.items()}
        self._instrument_names_int_to_str = {int(i): name for i, name in self.instrument_names.items()}
        
        self._bin_str_to_int = {name: int(i) for i, name in enumerate(self.bin_instrument_names)}

        self._bin_int_to_instrument_int = [self._instrument_names_str_to_int[self.bin_name_to_program_name[name]] if name != self.ch10_instrument_bin_name else 0 for name in self.bin_instrument_names]
        self._instrument_int_to_bin_int = [self._bin_str_to_int[self.program_name_to_bin_name[instr]] if self.program_name_to_bin_name[instr] != "" else -1 for instr in self.program_name_to_bin_name.keys()]

        self._ch10_bin_int = self._bin_str_to_int[self.ch10_instrument_bin_name] if self.ch10_instrument_bin_name else -1

        self.short_instr_bin_names = []
        for instr in self.bin_instrument_names:
            i = min(1, len(instr))
            while instr[:i] in self.short_instr_bin_names:
                i += 1
            self.short_instr_bin_names.append(instr[:i])
        self._short_instrument_names_str_to_int = {name: int(i) for i, name in enumerate(self.short_instr_bin_names)}

        range_excluding_ch10 = [(i if i < 9 else i+1) for i in range(len(self.bin_instrument_names))]
        bins_excluding_ch10 = [n for n in self.bin_instrument_names if n != self.ch10_instrument_bin_name]
        self.bin_channel_map = {bin: channel for channel, bin in zip(range_excluding_ch10, bins_excluding_ch10)}
        if self.ch10_instrument_bin_name:
            self.bin_channel_map[self.ch10_instrument_bin_name] = 9

    def validate(self):
        if self.max_wait_time % self.wait_events != 0:
            raise ValueError("max_wait_time must be exactly divisible by wait_events")
        if self.velocity_bins < 2:
            raise ValueError("velocity_bins must be at least 2")
        if len(self.bin_instrument_names) > 16:
            raise ValueError("bin_instruments must have at most 16 values")
        if self.velocity_bins_override:
            print("VocabConfig is using velocity_bins_override. Ignoring velocity_exp.")
            if len(self.velocity_bins_override) != self.velocity_bins:
                raise ValueError("velocity_bins_override must have same length as velocity_bins")
        if self.ch10_instrument_bin_name and self.ch10_instrument_bin_name not in self.bin_instrument_names:
            raise ValueError("ch10_instrument_bin_name must be in bin_instruments")
        if self.velocity_exp <= 0:
            raise ValueError("velocity_exp must be greater than 0")

    @classmethod
    def from_json(cls, path: str):
        with open(path, "r") as f:
            config = json.load(f)
        return cls(**config)


class VocabUtils:
    def __init__(self, cfg: VocabConfig) -> None:
        self.cfg = cfg
    
    @lru_cache(maxsize=128)
    def format_wait_token(self, wait: int) -> str:
        return f"t{wait}"

    @lru_cache(maxsize=128)
    def format_note_token(self, instrument_bin: int, note: int, velocity_bin: int) -> str:
        return f"{self.cfg.short_instr_bin_names[instrument_bin]}:{note:x}:{velocity_bin:x}"

    def format_unrolled_note(self, note: int) -> str:
        return f"n{note:x}"
    
    def format_unrolled_velocity(self, velocity_bin: int) -> str:
        return f"v{velocity_bin:x}"
    
    def format_unrolled_instrument_bin(self, instrument_bin: int) -> str:
        return f"i{self.cfg.short_instr_bin_names[instrument_bin]}"

    def velocity_to_bin(self, velocity: float) -> int:
        velocity = max(0, min(velocity, self.cfg.velocity_events - 1))
        if self.cfg.velocity_bins_override:
            for i, v in enumerate(self.cfg.velocity_bins_override):
                if velocity <= v:
                    return i
            return 0
        binsize = self.cfg.velocity_events / (self.cfg.velocity_bins - 1)
        if self.cfg.velocity_exp == 1.0:
            return ceil(velocity / binsize)
        else:
            return ceil((self.cfg.velocity_events*((self.cfg.velocity_exp**(velocity/self.cfg.velocity_events)-1.0) / (self.cfg.velocity_exp-1.0))) / binsize)

    def bin_to_velocity(self, bin: int) -> int:
        if self.cfg.velocity_bins_override:
            return self.cfg.velocity_bins_override[bin]
        binsize = self.cfg.velocity_events / (self.cfg.velocity_bins - 1)
        if self.cfg.velocity_exp == 1.0:
            return max(0, ceil(bin * binsize - 1))
        else:
            return max(0, ceil(self.cfg.velocity_events*log(((self.cfg.velocity_exp-1)*binsize*bin)/self.cfg.velocity_events+1, self.cfg.velocity_exp) - 1))

class AugmentValues:
    instrument_bin_remap: Dict[int, int]
    velocity_mod_factor: float
    transpose_semitones: int
    time_stretch_factor: float

    @classmethod
    def default(cls) -> "AugmentValues":
        return cls(
            instrument_bin_remap={},
            velocity_mod_factor=1.0,
            transpose_semitones=0,
            time_stretch_factor=1.0,
        )

class AugmentConfig:
    # The number of times to augment each MIDI file. The dataset size will be multiplied by this number.
    augment_data_factor: int
    # A list of instrument names to randomly swap with each other.
    instrument_mixups: List[List[str]]
    # A list of percentages to change the note velocity by. 0.0 = no change. 0 is included by default.
    velocity_mod_pct: List[float]
    # A list of semitones to transpose by. 0 is included by default.
    transpose_semitones: List[int]
    # A list of percentages to stretch the tempo by. 0.0 = no stretch. 0 is included by default.
    time_stretch_pct: List[float]
    # Random seed to use for reproducibility.
    seed: int

    cfg: VocabConfig

    def __post_init__(self):
        self.validate()
        if len(self.velocity_mod_pct) == 0:
            self.velocity_mod_pct = [0.0]
        if len(self.transpose_semitones) == 0:
            self.transpose_semitones = [0]
        if len(self.time_stretch_pct) == 0:
            self.time_stretch_pct = [0.0]
        
        self._instrument_mixups_int = [[self.cfg._bin_str_to_int[i] for i in l if i in self.cfg._bin_str_to_int] for l in self.instrument_mixups]
        self._instrument_mixups_int = [l for l in self._instrument_mixups_int if len(l) > 0]  # remove empty lists
        self._instrument_pool_assignments = {}
        self._mixup_pools = []
        for pool_i, mixup_list in enumerate(self._instrument_mixups_int):
            pool = set()
            for i in mixup_list:
                pool.add(i)
                self._instrument_pool_assignments[i] = pool_i
            self._mixup_pools.append(pool)


    def validate(self):
        if self.augment_data_factor < 1:
            raise ValueError("augment_data_factor must be at least 1")
        used_instruments = set()
        for mixup_list in self.instrument_mixups:
            for n in mixup_list:
                if n in used_instruments:
                    raise ValueError(f"Duplicate instrument name: {n}")
                used_instruments.add(n)

    @classmethod
    def from_json(cls, path: str, cfg: VocabConfig):
        with open(path, "r") as f:
            config = json.load(f)
        config["cfg"] = cfg
        if "seed" not in config:
            config["seed"] = random.randint(0, 2**32 - 1)
        return cls(**config)
    
    def get_augment_values(self, filename: str) -> Iterator[AugmentValues]:
        # first yield default values
        yield AugmentValues.default()

        rng = random.Random(self.seed + hash(filename))
        for _ in range(int(self.augment_data_factor - 1)):
            # randomize order for each pool
            randomized_pools = [list(pool) for pool in self._mixup_pools]
            for pool in randomized_pools:
                rng.shuffle(pool)
            # distribute reassignments
            instrument_bin_remap = {}
            for i, pool in enumerate(randomized_pools):
                for j, instrument in enumerate(pool):
                    instrument_bin_remap[instrument] = randomized_pools[i - 1][j]
            yield AugmentValues(
                instrument_bin_remap=instrument_bin_remap,
                velocity_mod_factor=1.0 + rng.choice(self.velocity_mod_pct),
                transpose_semitones=rng.choice(self.transpose_semitones),
                time_stretch_factor=1.0 + rng.choice(self.time_stretch_pct),
            )


class FilterConfig:
    # Whether to filter out MIDI files with duplicate MD5 hashes.
    deduplicate_md5: bool
    # Minimum time delay between notes in a file before splitting into multiple documents.
    piece_split_delay: float
    # Minimum length of a piece in milliseconds.
    min_piece_length: float

    @classmethod
    def from_json(cls, path: str):
        with open(path, "r") as f:
            config = json.load(f)
        return cls(**config)


In [19]:
def convert_midi_to_str(cfg: VocabConfig, filter_cfg: FilterConfig, mid: mido.MidiFile, augment: AugmentValues = None) -> List[str]:
    utils = VocabUtils(cfg)
    if augment is None:
        augment = AugmentValues.default()

    # filter out unknown meta messages before merge (https://github.com/mido/mido/pull/286)
    for i in range(len(mid.tracks)):
        mid.tracks[i] = [msg for msg in mid.tracks[i] if msg.type != "unknown_meta"]

    if len(mid.tracks) > 1:
        mid.tracks = [mido.merge_tracks(mid.tracks)]

    delta_time_ms = 0.0
    tempo = 500000
    channel_program = {i: 0 for i in range(16)}
    channel_volume = {i: 127 for i in range(16)}
    channel_expression = {i: 127 for i in range(16)}  # unlikely to be useful. expression usually modifies an already played note.
    channel_notes = {i: {} for i in range(16)}
    channel_pedal_on = {i: False for i in range(16)}
    channel_pedal_events = {i: {} for i in range(16)}  # {channel: {(note, program) -> True}}
    started_flag = False

    output_list = []
    output = ["<start>"]
    output_length_ms = 0.0
    token_data_buffer: List[Tuple[int, int, int, float]] = []  # need to sort notes between wait tokens

    def flush_token_data_buffer():
        nonlocal token_data_buffer, output, cfg, utils, augment
        token_data = [x for x in utils.prog_data_list_to_token_data_list(token_data_buffer)]
        if augment.instrument_bin_remap or augment.transpose_semitones:
            # TODO put transpose in a real function
            raw_transpose = lambda bin, n: n + augment.transpose_semitones if bin != cfg._ch10_bin_int else n
            octave_shift_if_oob = lambda n: n + 12 if n < 0 else n - 12 if n >= cfg.note_events else n
            # TODO handle ranges beyond 12
            #octave_shift_if_oob = lambda n: 0 if n < 0 else (n - cfg.note_events) % 12 + cfg.note_events if n >= cfg.note_events else n
            transpose = lambda bin, n: octave_shift_if_oob(raw_transpose(bin, n))

            token_data = [(augment.instrument_bin_remap.get(i, i), transpose(i, n), v) for i, n, v in token_data]
        if cfg.do_token_sorting:
            token_data = utils.sort_token_data(token_data)
        if cfg.unrolled_tokens:
            for t in token_data:
                output += [utils.format_unrolled_instrument_bin(t[0]), utils.format_unrolled_note(t[1]), utils.format_unrolled_velocity(t[2])]
        else:
            output += [utils.format_note_token(*t) for t in token_data]
        token_data_buffer = []

    def consume_note_program_data(prog: int, chan: int, note: int, vel: float):
        nonlocal output, output_length_ms, started_flag, delta_time_ms, cfg, utils, token_data_buffer
        is_token_valid = utils.prog_data_to_token_data(prog, chan, note, vel) is not None
        if not is_token_valid:
            return
        
        if delta_time_ms > filter_cfg.piece_split_delay * 1000.0:
            # check if any notes are still held
            silent = True
            for channel in channel_notes.keys():
                if len(channel_notes[channel]) > 0:
                    silent = False
                    break
            if silent:
                flush_token_data_buffer()
                output.append("<end>")
                if output_length_ms > filter_cfg.min_piece_length * 1000.0:
                    output_list.append(" ".join(output))
                output = ["<start>"]
                output_length_ms = 0.0
                started_flag = False
        if started_flag:
            wait_tokens = utils.data_to_wait_tokens(delta_time_ms)
            if len(wait_tokens) > 0:
                flush_token_data_buffer()
                output_length_ms += delta_time_ms
                output += wait_tokens
        delta_time_ms = 0.0
        token_data_buffer.append((prog, chan, note, vel * augment.velocity_mod_factor))
        started_flag = True

    for msg in mid.tracks[0]:
        time_ms = mido.tick2second(msg.time, mid.ticks_per_beat, tempo) * 1000.0
        delta_time_ms += time_ms
        t = msg.type

        if msg.is_meta:
            if t == "set_tempo":
                tempo = msg.tempo * augment.time_stretch_factor
            continue

        def handle_note_off(ch, prog, n):
            if channel_pedal_on[ch]:
                channel_pedal_events[ch][(n, prog)] = True
            else:
                consume_note_program_data(prog, ch, n, 0)
                if n in channel_notes[ch]:
                    del channel_notes[ch][n]

        if t == "program_change":
            channel_program[msg.channel] = msg.program
        elif t == "note_on":
            if msg.velocity == 0:
                handle_note_off(msg.channel, channel_program[msg.channel], msg.note)
            else:
                if (msg.note, channel_program[msg.channel]) in channel_pedal_events[msg.channel]:
                    del channel_pedal_events[msg.channel][(msg.note, channel_program[msg.channel])]
                consume_note_program_data(
                    channel_program[msg.channel],
                    msg.channel,
                    msg.note,
                    mix_volume(msg.velocity, channel_volume[msg.channel], channel_expression[msg.channel]),
                )
                channel_notes[msg.channel][msg.note] = True
        elif t == "note_off":
            handle_note_off(msg.channel, channel_program[msg.channel], msg.note)
        elif t == "control_change":
            if msg.control == 7 or msg.control == 39:  # volume
                channel_volume[msg.channel] = msg.value
            elif msg.control == 11:  # expression
                channel_expression[msg.channel] = msg.value
            elif msg.control == 64:  # sustain pedal
                channel_pedal_on[msg.channel] = msg.value >= 64
                if not channel_pedal_on[msg.channel]:
                    for (note, program) in channel_pedal_events[msg.channel]:
                        handle_note_off(msg.channel, program, note)
                    channel_pedal_events[msg.channel] = {}
            elif msg.control == 123:  # all notes off
                for channel in channel_notes.keys():
                    for note in list(channel_notes[channel]).copy():
                        handle_note_off(channel, channel_program[channel], note)
        else:
            pass

    flush_token_data_buffer()
    output.append("<end>")
    if output_length_ms > filter_cfg.min_piece_length * 1000.0:
        output_list.append(" ".join(output))
    return output_list


In [20]:
cfg = VocabConfig()
filter_cfg = FilterConfig()

mid = mido.MidiFile(midi_files[0])
text = '\n'.join(convert_midi_to_str(cfg, filter_cfg, mid))
    

AttributeError: 'AugmentValues' object has no attribute 'time_stretch_factor'

REMI TOKENIZER USING MIDITOK , https://github.com/Natooz/MidiTok